In [4]:
import syft as sy

In [8]:
duet = sy.join_duet(loopback=True)

🎤  🎸  ♪♪♪ Joining Duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently in beta.
♫♫♫ > Use at your own risk.


    > ❤️ Love Duet? Please consider supporting our community!
    > https://github.com/sponsors/OpenMined

♫♫♫ > Punching through firewall to OpenGrid Network Node at:
♫♫♫ > http://ec2-18-218-7-180.us-east-2.compute.amazonaws.com:5000
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... 
♫♫♫ > DONE!


Exception in callback Transaction.__retry()
handle: <TimerHandle when=543717.046 Transaction.__retry()>
Traceback (most recent call last):
  File "C:\Users\21355\Anaconda3\envs\pysyft\lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\21355\Anaconda3\envs\pysyft\lib\site-packages\aioice\stun.py", line 306, in __retry
    self.__future.set_exception(TransactionTimeout())
  File "C:\Users\21355\Anaconda3\envs\pysyft\lib\asyncio\futures.py", line 270, in set_exception
    raise exceptions.InvalidStateError(f'{self._state}: {self!r}')
asyncio.exceptions.InvalidStateError: FINISHED: <Future finished result=(Message(messa...8w\xb7S\xfe}'), ('192.168.56.1', 63893))>



♫♫♫ > CONNECTED!


In [9]:
duet.store.pandas

,ID,Tags,Description,object_type
0,<UID: ca59b8f1f288423487084b9f50fd83d7>,[Dataset],This is a dummy data for the exemple,<class 'torch.Tensor'>


on va crée notre model 

In [10]:
class SyNet(sy.Module): 
    def __init__(self,torch_ref): 
        super(SyNet, self).__init__(torch_ref=torch_ref)
        self.lin = self.torch_ref.nn.Linear(2,2)
        
    def forward(self,x): 
        x = self.lin(x)
        return x

In [11]:
import torch 

In [12]:
model = SyNet(torch)

In [13]:
model_pointer = model.send(duet)

In [14]:
params = model_pointer.parameters()
optim = duet.torch.optim.SGD(params=params,lr=0.1)

In [15]:
data_pointer = duet.store[0]

mtn on va crée le 2eme model qui va etre local au niveau du DS et mm les donnée qui seront les labels

In [16]:
model_2 = torch.nn.Linear(2,1)

optim2 = torch.optim.SGD(params=model_2.parameters(),lr=0.1)

target= torch.tensor([[0.0],[1.0],[0.0],[0.0]])

In [18]:
for iter in range(20): 
    #setting the gradients of the optimizers to ZERO 
    optim.zero_grad()
    optim2.zero_grad()
    
    #calculate the output of the remote model 
    activation_pointer = model_pointer(data_pointer)
    activation = activation_pointer.clone().get(request_block=True)
    #we used the .clone() for not loosing the reference for the activation pointer on the computations('')
    pred = model_2(activation)

    loss = ((pred - target)**2).sum()
    loss.backward()

    grad_pointer = activation.grad.clone().send(duet)
    activation_pointer.backward(grad_pointer)
    
    optim.step()
    optim2.step()
    
    print(loss)

tensor(0.2511, grad_fn=<SumBackward0>)
tensor(0.2507, grad_fn=<SumBackward0>)
tensor(0.2505, grad_fn=<SumBackward0>)
tensor(0.2503, grad_fn=<SumBackward0>)
tensor(0.2502, grad_fn=<SumBackward0>)
tensor(0.2502, grad_fn=<SumBackward0>)
tensor(0.2501, grad_fn=<SumBackward0>)
tensor(0.2501, grad_fn=<SumBackward0>)
tensor(0.2501, grad_fn=<SumBackward0>)
tensor(0.2500, grad_fn=<SumBackward0>)
tensor(0.2500, grad_fn=<SumBackward0>)
tensor(0.2500, grad_fn=<SumBackward0>)
tensor(0.2500, grad_fn=<SumBackward0>)
tensor(0.2500, grad_fn=<SumBackward0>)
tensor(0.2500, grad_fn=<SumBackward0>)
tensor(0.2500, grad_fn=<SumBackward0>)
tensor(0.2500, grad_fn=<SumBackward0>)
tensor(0.2500, grad_fn=<SumBackward0>)
tensor(0.2500, grad_fn=<SumBackward0>)
tensor(0.2500, grad_fn=<SumBackward0>)


In [19]:
type(data_pointer)

syft.proxy.torch.TensorPointer

In [69]:
duet.store.pandas

,ID,Tags,Description,object_type
0,<UID: 7874920264f04a898d96fa687c0c7cbb>,[Dataset],This is a dummy data for the exemple,<class 'torch.Tensor'>
1,<UID: b66da672147f4d47af508ef2a4b0bda4>,[],,<class 'torch.Tensor'>
2,<UID: 48df3785510347eab88025a37b427bcc>,[],,<class 'torch.Tensor'>
3,<UID: a92cf47b7e83452da6aebdd6ec56437d>,[],,<class 'torch.Tensor'>
4,<UID: 212579c796484b5d8c6a11360d767d32>,[],,<class 'torch.Tensor'>
...,...,...,...,...
99,<UID: 1996f79e1ff242f09c61f401e43511a4>,[],,<class 'torch.Tensor'>
100,<UID: a0a1ba44675541cd97623d5b2b83a284>,[],,<class 'torch.Tensor'>
101,<UID: c27718431cd2482f82d75b0871c12a98>,[],,<class 'torch.Tensor'>
102,<UID: d9955c7dc9b34e19a1d693de360356b2>,[],,<class 'torch.Tensor'>


In [59]:
activation_pointer = model_pointer(data_pointer)
activation = activation_pointer.clone().get(request_block=True)
#we used the .clone() for not loosing the reference for the activation pointer on the computations('')

pred = model_2(activation)

loss = ((pred - target)**2).sum()
loss.backward()

print(pred)
grad_pointer = activation.grad.clone().send(duet)
activation_pointer.backward(grad_pointer)

optim.step()
optim2.step()

print(loss)

tensor([[0.2748],
        [0.2895],
        [0.1955],
        [0.1809]], grad_fn=<AddmmBackward>)
tensor(0.6514, grad_fn=<SumBackward0>)


In [61]:
activation_pointer = model_pointer(data_pointer)
activation = activation_pointer.clone().get(request_block=True)
#we used the .clone() for not loosing the reference for the activation pointer on the computations('')

pred = model_2(activation)

loss = ((pred - target)**2).sum()
loss.backward()

print(pred)
grad_pointer = activation.grad.clone().send(duet)
activation_pointer.backward(grad_pointer)

optim.step()
optim2.step()

print(loss)

tensor([[0.4244],
        [0.5709],
        [0.3938],
        [0.2472]], grad_fn=<AddmmBackward>)
tensor(0.5804, grad_fn=<SumBackward0>)


In [62]:
activation_pointer = model_pointer(data_pointer)
activation = activation_pointer.clone().get(request_block=True)
#we used the .clone() for not loosing the reference for the activation pointer on the computations('')

pred = model_2(activation)

loss = ((pred - target)**2).sum()
loss.backward()

print(pred)
grad_pointer = activation.grad.clone().send(duet)
activation_pointer.backward(grad_pointer)

optim.step()
optim2.step()

print(loss)

tensor([[0.3989],
        [0.6404],
        [0.3963],
        [0.1548]], grad_fn=<AddmmBackward>)
tensor(0.4694, grad_fn=<SumBackward0>)
